# MIT 6.036 Spring 2019: Homework 12

In [2]:
import numpy as np
import time
import pickle

# Setup
Run the next code block to download and import the code for this lab.



In [7]:
!wget --no-check-certificate https://introml_oll.odl.mit.edu/cat-soop/_static/6.036/homework/hw12/code_and_data_for_hw12.zip
!unzip code_and_data_for_hw12.zip

--2023-04-20 10:47:57--  https://introml_oll.odl.mit.edu/cat-soop/_static/6.036/homework/hw12/code_and_data_for_hw12.zip
Resolving introml_oll.odl.mit.edu (introml_oll.odl.mit.edu)... 3.226.240.108
Connecting to introml_oll.odl.mit.edu (introml_oll.odl.mit.edu)|3.226.240.108|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6963985 (6.6M) [application/zip]
Saving to: ‘code_and_data_for_hw12.zip’

code_and_data_for_h 100%[===================>]   6.64M  5.78MB/s    in 1.1s    

2023-04-20 10:47:59 (5.78 MB/s) - ‘code_and_data_for_hw12.zip’ saved [6963985/6963985]

Archive:  code_and_data_for_hw12.zip
  inflating: hw12_code_and_data/code_for_hw12.py  
  inflating: hw12_code_and_data/movies.csv  
  inflating: hw12_code_and_data/ratings.csv  


# 1) Matrix factorization by rank
Consider the following data set:

$$
Y =
\begin{bmatrix}
    4 & 3 & 1 \\
    1 & 3 & -2 \\
    5 & 2 & 3 \\
\end{bmatrix}
$$

We would like to factorize $Y=UV^T$ such that $U$ and $V$ are both $3 x k$ matrices.

Hint: See if any columns are linear combinations of others. Try subtracting the first two columns.

### 1A)
Can this matrix be factored using $k=1$? Provide $U$, $V^T$ matrices if it can (such that $Y=UV^T$), or enter 'None' if it cannot be done.

### 1B)
Can this matrix be factored using $k=2$? Provide $U$, $V^T$ matrices if it can (such that $Y=UV^T$), or enter 'None' if it cannot be done.

In [90]:
Y = np.array([[4, 3, 1], [1, 3, -2], [5, 2, 3]])
U = np.array([[4, 3], [1, 3], [5, 2]])
V = np.array([[1, 0, 1], [0, 1, -1]]).T
U@V.T

array([[ 4,  3,  1],
       [ 1,  3, -2],
       [ 5,  2,  3]])

# Some preliminary code

Here are some useful functions which will be used in the following sections:

In [14]:
def pred(data, x):
    (a, i, r) = data
    (u, b_u, v, b_v) = x
    return np.dot(u[a].T, v[i]) + b_u[a] + b_v[i]
  
# X : n x k
# Y : n
def ridge_analytic(X, Y, lam):
    (n, k) = X.shape
    xm = np.mean(X, axis = 0, keepdims = True)   # 1 x n
    ym = np.mean(Y)                              # 1 x 1
    Z = X - xm                                   # d x n
    T = Y - ym                                   # 1 x n
    th = np.linalg.solve(np.dot(Z.T, Z) + lam * np.identity(k), np.dot(Z.T, T))
    # th_0 account for the centering
    th_0 = (ym - np.dot(xm, th))                 # 1 x 1
    return th.reshape((k,1)), float(th_0)

### Lab reference
Note that in practice, it is impossible to have all users rate all movies; companies like Netflix recommend movies to its users who have rated only a subset of movies.

To make this concrete, consider an incomplete version of the data from the previous question:

$$Y = \begin{bmatrix}
? & 1 & ? & 1 & 5 & 1 & 5 \\
1 & 5 & 1 & ? & ? & 5 & 1 \\
5 & 5 & 5 & 5 & ? & ? & 5 \\
1 & ? & 1 & 1 & 1 & 1 & ? 
\end{bmatrix}$$
 
Now our goal is to find $U$, $V^T$ that minimizes $J(U,V)$ given the incomplete matrix $Y$ above. To do so, we will use the __alternating least squares__ method, where we

1. Fix (or initialize) $V$ and find the optimal $U$ that minimizes $J(U)$ given this $V$.
2. Fix $U$ to the value obtained from 1, and find the optimal $V$ that minimizes $J(V)$ given this $U$.
3. Repeat from 1, with $V$ fixed to value obtained from 2.

To implement the alternating least squares method with our incomplete matrix $Y$, we will define some notation that will make the gradient $\nabla_{u^{(a)}} J(U)$ easier to write in matrix form, and similarly for $\nabla_{v^{(i)}} J(V)$.

Let $I_a$ be the set of all movies that were rated by person $a$. To be precise, $I_a = \{i | (a, i) \in D\}$. Also, define $B_a$ to be the matrix of relevant data: $B_a$ is constructed by taking $V$ and removing every row $i$ that is not in $I_a$. In other words, $B_a$ is the matrix of movie features for just the movies that were rated by person $a$. Letting $l_a$ be the number of elements in $I_a$, observe that $B_a$ has dimensions $l_a$ by $k$.

### End Lab reference

## 2) Some movies are more equal than others

In Lab 12, we formulated recommender systems without offsets:  there was no equivalent of $\theta_0$ in the regression problem we solved.  But offsets are very useful in this problem to account for some reviewers that are just generally grumpy (or enthusiastic) or some movies that are just generally awful (or great). For this section we will work to extend the results from the lab to include offsets.

Rather than trying to predict what rating Amy will give a movie based on the properties of that movie, it might be more effective to predict how much "more highly than usual" Amy will rate the movie.  To do this, we introduce a vector $b_u$ to characterize offsets for users and a vector $b_v$ to characterize offsets for movies.  Now, the objective can be written as
$$J(U, V) = \frac{1}{2}\sum_{(a, i) \in D} (Y_{ai} - {u^{(a)}}\cdot v^{(i)} - b_u^{(a)} - b_v^{(i)})^2 + \frac{\lambda}{2} \sum_{a = 1}^n \lVert u^{(a)} \rVert^2 + \frac{\lambda}{2} \sum_{i = 1}^m  \lVert v^{(i)} \rVert^2 $$

We will stick with just one half of the problem for now:  finding $U$ and $b_u$ given a fixed $V$ and $b_v$.  We'll concentrate on computing a new estimate for $u^{(a)}$ and $b_u^{(a)}$.  Continuing the notation from lab, the regression problem that we have to solve becomes:
$$-B_a^T (Z_a - B_a u^{(a)} - b_v - b_u^{(a)}) + \lambda u^{(a)} = 0$$
where $b_v$ is the $l_a$ by 1 vector of offsets for the movies and $b_u^{(a)}$ is the offset for user $a$.  This is a linear regression problem in which the targets can be viewed as $Z_a - b_v$, and where there is an offset $b_u^{(a)}$ that is not regularized.  This is, then, the standard ridge regression set-up.  We have seen how to solve this problem via gradient descent before, but it can also be solved analytically by essentially turning it into a regular linear regression problem. Using this approach to computing the optimal $u^{(a)}$ requires first centering the data, then doing linear regression, then doing a little bit of work to recover the offset $b_u^{(a)}$.  In the code, we have supplied a procedure (`ridge_analytic`) for doing exactly this.

Let's see how it works out in our running example.  For illustrative purposes, we'll add two more movies:

8) 6.036 lecture videos <br />
9) The Zzzz files <br />

Our new preference data is:
$$Y = \begin{bmatrix}
? & 1 & ? & 1 & 5 & 1 & 5 & 5 & 1\\
1 & 5 & 1 & ? & ? & 5 & 1 & 5 & 1\\
5 & 5 & 5 & 5 & ? & ? & 5 & ? & 1\\
1 & ? & 1 & 1 & 1 & 1 & ? & 5 & ?
\end{bmatrix}$$

We will assume the following value for $b_v$:
$$b_v = \begin{bmatrix}
3 & 3 & 3 & 3 & 3 & 3 & 3 & 5 & 1
\end{bmatrix}^T $$
And let
$$V = \begin{bmatrix}
10 & 1 & 10 & 1 & 10 & 1 & 10 & 5 & 5\\
1 & 10 & 1 & 10 & 1 & 10 & 1 & 5 & 5
\end{bmatrix}
$$
We can now compute the optimal $u^{(a)}$ and $b_u^{(a)}$ for $a=0$:

In [92]:
Z = np.array([[1], [1], [5], [1], [5], [5], [1]])
b_v = np.array([[3], [3], [3], [3], [3], [5], [1]])
B = np.array([[1, 10], [1, 10], [10, 1], [1, 10], [10, 1], [5, 5], [5, 5]])
# Solution with offsets, using ridge_analytic provided in code file
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)

#u_a, b_u_a
#(array([[ 0.22024566],
#        [-0.22193986]]),
# array([[ 0.00762389]]))

# Solution using previous model, with no offsets
u_a_no_b = np.dot(np.linalg.inv(np.dot(B.T, B) + 1 * np.identity(2)), np.dot(B.T, Z))

#u_a_no_b
#array([[ 0.50148126],
#       [ 0.0562376 ]])

The code above computes $u^{(a)}$ and $b_u^{(a)}$for Amy. Use them for the next few parts. You can assume that Llama movies always satisfy $v^{(i)} = \begin{bmatrix} 10 \\ 1 \\ \end{bmatrix}$ and Robot movies satisfy $v^{(i)} = \begin{bmatrix} 1 \\ 10 \\ \end{bmatrix}$

### 2A)
How will Amy feel about a brand new Llama movie (not in the existing movie data) that gets bad ratings from almost everyone ($b_v^{(i)} = 1$), in the two models?

### 2B)
What about a brand new Robot movie (not in the existing movie data) that gets good ratings from almost everyone ($b_v^{(i)} = 5$), in the two models?

### 2C)
What about a brand new Robot movie (not in the existing movie data) that gets good ratings from almost everyone ($b_v^{(i)} = 3$), in the two models?

In [121]:
Z = np.array([[1], [1], [5], [1], [5], [5], [1]])
b_v = np.array([[1], [1], [1], [1], [1], [1], [1]])
B = np.array([[1, 10], [1, 10], [10, 1], [1, 10], [10, 1], [5, 5], [5, 5]])
# Solution with offsets, using ridge_analytic provided in code file
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)

# Solution using previous model, with no offsets
u_a_no_b = np.dot(np.linalg.inv(np.dot(B.T, B) + 1 * np.identity(2)), np.dot(B.T, Z))

v_llama = np.array([[10], [1]])
v_robot = np.array([[1], [10]])

print(np.round(np.dot(u_a_no_b.T, v_llama), 2), np.round(np.dot(u_a.T, v_llama) + b_u_a, 2))

b_v = np.array([[5], [5], [5], [5], [5], [5], [5]])
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)
print(np.round(np.dot(u_a_no_b.T, v_robot), 2), np.round(np.dot(u_a.T, v_robot) + b_u_a, 2))

b_v = np.array([[3], [3], [3], [3], [3], [3], [3]])
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)
print(np.round(np.dot(u_a_no_b.T, v_robot), 2), np.round(np.dot(u_a.T, v_robot) + b_u_a, 2))

[[5.07]] [[3.99]]
[[1.06]] [[-3.99]]
[[1.06]] [[-1.99]]


# 3) Implementing recommender systems

Now we'll look in detail at two implementations of matrix factorization.  One is the alternating least squares algorithm, and the other is the stochastic gradient descent algorithm.

We will assume that the input data is made up of `(a, i, r)` triples, where `a` is a user index. `i` is an item index and `r` is a rating.  Here is a small example:

In [15]:
# Data is a list of (a, i, r) triples
ratings_small = \
[(0, 0, 5), (0, 1, 3), (0, 3, 1),
 (1, 0, 4), (1, 3, 1),
 (2, 0, 1), (2, 1, 1), (2, 3, 5),
 (3, 0, 1), (3, 3, 4),
 (4, 1, 1), (4, 2, 5), (4, 3, 4)]

We will assume that predictions are made using user and item offsets, that is,
$$y = {u^{(a)}}\cdot v^{(i)} + b_u^{(a)} + b_v^{(i)}$$
In this expression $b_u$ is a vector of user offsets and $b_v$ is a vector of item offsets.

### 3.1) Alternating Least Squares (ALS)
Below is a function that provides the "outer loop" of the alternating least squares algorithm.

* Define n and m from the data.
* Initialize a list of lists `us_from_v` where `us_from_v[i]` contains the indices and ratings of users who rated item `i`.
* Similarly, initialize a list of lists `vs_from_u` where `vs_from_u[a]` contains the indices and ratings of items rated by user `a`.
* Initialize the set of parameters `x` (note that the u,v entries are set randomly while the user and item offsets are set to 0) where 
    * `x[0] = u`, a list of column vectors (initialized randomly) such that `u[a]` corresponds to $u^{(a)}$ as defined above.
    * `x[1] = b_u`, a column vector (initialized with 0s) equal to $b_u$ as defined above.
    * `x[2] = v`, a list of column vectors (initialized randomly) such that `v[i]` corresponds to $v^{(i)}$ as defined above.
    * `x[3] = b_v`, a column vector (initialized with 0s) equal to $b_v$ as defined above.  
* Then we alternate minimizations.
* And report the results: the error between predicted scores and a held-out set of actual scores on the same users and items.


In [16]:
# The ALS outer loop
def mf_als(data_train, data_validate, k=2, lam=0.02, max_iter=100, verbose=False):
    # size of the problem
    n = max(d[0] for d in data_train)+1 # users
    m = max(d[1] for d in data_train)+1 # items
    # which entries are set in each row and column
    us_from_v = [[] for i in range(m)]
    vs_from_u = [[] for a in range(n)]
    for (a, i, r) in data_train:
        # item i was rated by users a with rating r
        us_from_v[i].append((a, r))
        # user a rated items i with rating r
        vs_from_u[a].append((i, r))
    # Initial guess at u, b_u, v, b_v
    # Note that u and v are lists of column vectors (columns of U, V).
    x = ([np.random.normal(1/k, size=(k,1)) for a in range(n)],
          np.zeros(n),
          [np.random.normal(1/k, size=(k,1)) for i in range(m)],
          np.zeros(m))
    # Alternation, modifies the contents of x
    start_time = time.time()
    for i in range(max_iter):
        update_U(data_train, vs_from_u, x, k, lam)
        update_V(data_train, us_from_v, x, k, lam)
        if verbose:
            print('train rmse', rmse(data_train, x), 'validate rmse', data_validate and rmse(data_validate, x))
        if data_validate == None: # code is slower, print out progress
            print("Iteration {} finished. Total Elapsed Time: {:.2f}".format(i + 1, time.time() - start_time))
    # The root mean square errors measured on validate set
    if data_validate != None:
        print('ALS result for k =', k, ': rmse train =', rmse(data_train, x), '; rmse validate =', rmse(data_validate, x))
    return x

The function `ridge_analytic(X,Y,lam)` is defined at the top of the file. Furthermore, the above code computes RMSE via the following function:


In [17]:
# Compute the root mean square error
def rmse(data, x):
    error = 0.
    for datum in data:
        error += (datum[-1] - pred(datum, x))**2
    return np.sqrt(error/len(data))

Here is an example of `vs_from_u` for the small data set given above:

```
vs_from_u = \
[[(0, 5), (1, 3), (3, 1)],
 [(0, 4), (3, 1)],
 [(0, 1), (1, 1), (3, 5)],
 [(0, 1), (3, 4)],
 [(1, 1), (2, 5), (3, 4)]]
```



Now, the only part that's missing are the `update_U` and `update_V`
procedures.  These are very similar, so we'll just do `update_U`,
where we hold the $v^{(i)}$ constant and solve for the $u^{(a)}$.  We
have seen above that each of the steps is solving a ridge regression
problem, that is, finding a set of coefficients for a linear function,
$(\theta, \theta_0)$, so as to minimize the mean sum of squared errors
on data given by $(X,Y)$ (with regularization on the magnitude of
$\theta$).

We have given you a function `ridge_analytic(X,Y,lam)` that solves the ridge regression problem and returns `(th, th0)` as usual, where `th` is a column vector and `th0` is a float.

### 3.1)
Now, write the function `update_U(data, vs_from_u, x, k, lam)`

* `data` is a list of `(a, i, r)` triples
* `vs_from_u` is a list of lists as defined above
* `x` is a list of parameters as defined above
* `k` is an integer indicating the length of the individual u and v vectors
* `lam` is the regularization parameter

The function should update the entries in `x` corresponding to the `u` vectors and the `b_u` entries.  It should also return `x`, so the Tutor can check it.
Note that if there are no ratings from a particular user, we don't want to update that user's entries.


In [18]:
def update_U(data, vs_from_u, x, k, lam):
    (u, b_u, v, b_v) = x
    # Your code here
    # user a rated items i with rating r
    # vs_from_u[a].append((i, r))
    n = len(vs_from_u)
    for a in range(n):
        # equivalent lab notation in comments
        if not vs_from_u[a]:
            continue
        items_user_a_rated = vs_from_u[a]
        # B_a, l_a x k
        v_i = np.hstack([v[i] for i, _ in items_user_a_rated]).T
        # Z_a, l_a x 1
        y_ai = np.array([r for _, r in items_user_a_rated]).reshape((-1, 1))
        b_v_i = b_v[[i for i, _ in items_user_a_rated]].reshape((-1, 1))
        u_a, b_u_a = ridge_analytic(v_i, (y_ai - b_v_i), lam)
        x[0][a] = u_a
        x[1][a] = b_u_a
    return x


Here is a function to help test your code. It uses `ratings_small`, defined above.

In [63]:
def update_U_test():
    '''
    This is a test function provided to help you debug your implementation
    '''
    k = 2
    lam = 0.01
    
    vs_from_u = \
      [[(0, 5), (1, 3), (3, 1)],
       [(0, 4), (3, 1)],
       [(0, 1), (1, 1), (3, 5)],
       [(0, 1), (3, 4)],
       [(1, 1), (2, 5), (3, 4)]]
  
    np.random.seed(0)
    
    first = []
    for i in range(5):
        first.append(np.random.rand(2, 1))
    second = np.zeros((5,))
    third = []
    for i in range(5):
        third.append(np.random.rand(2, 1))
    fourth = np.zeros((5,))
    x0 = (first, second, third, fourth)
    
    x_result = update_U(ratings_small, vs_from_u, x0, k, lam)
    
    assert np.all(np.isclose(x_result[0], np.array([[[4.048442188078757], [-2.5000082235465526]],
                                                [[3.2715388359271054], [-1.2879317400952521]],
                                                [[-6.237522315961142], [-2.9639103597721355]],
                                                [[-3.2715388359271054], [1.2879317400952521]],
                                                [[-4.87111151185168], [-1.761023196019822]] ])))
    assert np.all(np.isclose(x_result[1].reshape(-1,), 
                           np.array([[3.043665230868208], [2.048616799474877], [7.462166369240114], [2.951383200525123], [5.487071919883842]]).reshape(-1,)))
    assert np.all(np.isclose(x_result[2], np.array([[[0.7917250380826646], [0.5288949197529045]],
                                       [[0.5680445610939323], [0.925596638292661]],
                                       [[0.07103605819788694], [0.08712929970154071]],
                                       [[0.02021839744032572], [0.832619845547938]],
                                       [[0.7781567509498505], [0.8700121482468192]] ])))
    assert np.all(np.isclose(x_result[3].reshape(-1,), np.array([[0.0], [0.0], [0.0], [0.0], [0.0]]).reshape(-1,)))
    print("Test passed!")
    
update_U_test()

Test passed!


**NOTE: ** The following **does not** need to be written: it can be filled in by clicking View Answer in the update_U problem. It will be useful in part 4.

In [27]:
def update_V(data, us_from_v, x, k, lam):
    (u, b_u, v, b_v) = x
    # Your code here
    # item i was rated by users a with rating r
    # us_from_v[i].append((a, r))
    m = len(us_from_v)
    for i in range(m):
        # equivalent lab notation in comments
        if not us_from_v[i]:
            continue
        item_i_ratings = us_from_v[i]
        # B_a, l_a x k
        u_a = np.hstack([u[a] for a, _ in item_i_ratings]).T
        # Z_a, l_a x 1
        y_ai = np.array([r for _, r in item_i_ratings]).reshape((-1, 1))
        b_u_a = b_u[[a for a, _ in item_i_ratings]].reshape((-1, 1))
        v_i, b_v_i = ridge_analytic(u_a, (y_ai - b_u_a), lam)
        x[2][i] = v_i
        x[3][i] = b_v_i
    return x

### 3.2) Stochastic Gradient Descent (SGD)
Alternatively, we can use Stochastic Gradient Descent directly on the objective function
$$J(U, V) = \frac{1}{2}\sum_{(a, i) \in D} (Y_{ai} - {u^{(a)}}\cdot v^{(i)} - b_u^{(a)} - b_v^{(i)})^2 + \frac{\lambda}{2} \sum_{a = 1}^n \lVert u^{(a)} \rVert^2 + \frac{\lambda}{2} \sum_{i = 1}^m  \lVert v^{(i)} \rVert^2 $$
Note, however, that this is not strictly a sum over the data.  The regularization terms are not inside the first summation and they cannot simply be moved inside, since we would end up penalizing the parameters unevenly depending on how many ratings there were for particular items by particular users.

We can, however, define vectors of values $\lambda_u^{(a)}$ and $\lambda_v^{(i)}$ so that this is equivalent to the original objective:
$$J(U, V) = \frac{1}{2}\sum_{(a, i) \in D} \left[(Y_{ai} - {u^{(a)}}\cdot v^{(i)} - b_u^{(a)} - b_v^{(i)})^2 + \lambda_u^{(a)}\lVert u^{(a)} \rVert^2 + \lambda_v^{(i)} \lVert v^{(i)} \rVert^2\right] $$

In order to make this work out, we must have had:
$$\frac{\lambda}{2} \sum_{a = 1}^n \lVert u^{(a)} \rVert^2 = \frac{1}{2}\sum_{(a, i) \in D} \lambda_u^{(a)}\lVert u^{(a)} \rVert^2 $$
matching the terms for each $a$ separately, we must have:
$$ \frac{\lambda}{2} \lVert u^{(a)} \rVert^2 = \sum_{i : (a, i) \in D} \frac{\lambda_u^{(a)}}{2} \lVert u^{(a)} \rVert^2 = \lvert \{i : (a, i) \in D\} \rvert \frac{\lambda_u^{(a)}}{2} \lVert u^{(a)} \rVert^2 $$
Then, we must have:
$$ \lambda_u^{(a)} = \frac{\lambda}{\lvert \{i : (a, i) \in D\} \rvert}$$
Similarly,
$$ \lambda_v^{(i)} = \frac{\lambda}{\lvert \{a : (a, i) \in D\} \rvert}$$


This is now strictly a sum over the data. Note that the offset terms are not regularized.

The "outer loop" of this approach is given here:

* Define n and m from the data.
* Define vectors $\lambda_u$ and $\lambda_v$.
* Initialize the set of parameters (created and initialized as in ALS).  
* Loop picking a random data entry `(a,i,r)` and taking a step down the gradient during each iteration.
* And report the results: the error between predicted scores and a held-out set of actual scores on the same users and items.

In [28]:
# The SGD outer loop
def mf_sgd(data_train, data_validate, step_size_fn, k=2, lam=0.02, max_iter=100, verbose=False):
    # size of the problem
    ndata = len(data_train)
    n = max(d[0] for d in data_train)+1
    m = max(d[1] for d in data_train)+1
    # Distribute the lambda among the users and items
    lam_uv = lam/counts(data_train,0), lam/counts(data_train,1)
    # Initial guess at u, b_u, v, b_v (also b)
    x = ([np.random.normal(1/k, size=(k,1)) for j in range(n)],
         np.zeros(n),
         [np.random.normal(1/k, size=(k,1)) for j in range(m)],
         np.zeros(m))
    di = int(max_iter/10.)
    for i in range(max_iter):
        if i%di == 0 and verbose:
            print('i=', i, 'train rmse=', rmse(data_train, x),
                  'validate rmse', data_validate and rmse(data_validate, x))
        step = step_size_fn(i)
        j = np.random.randint(ndata)            # pick data item
        sgd_step(data_train[j], x, lam_uv, step) # modify x
    print('SGD result for k =', k, ': rmse train =', rmse(data_train, x), '; rmse validate =', rmse(data_validate, x))
    return x

### 3.2)
The missing procedure is `sgd_step` which takes a gradient step using a
noisy estimate of the gradient based on one point.  This function will
update the relevant components of `x` corresponding to `u[a]`,
`b_u[a]`, `v[i]` and `b_v[i]`.  Your function should modify the entries in `x`
and also return `x` so that the Tutor can check it.

You will need to derive the gradient for each of the variable
components of `x`, including the offsets.  Refer back to section 1 for guidance, but
remember the offsets and remember that the offset terms are not regularized.

__WARNING: In numpy `x += y` and `x -= y` can produce different results from `x = x+y` and `x = x-y`.  For consistency in checking, please don't use `x += y` or `x -= y`! 
<a href="https://stackoverflow.com/questions/31987713/numpy-array-difference-between-a-x-vs-a-a-x">Read here</a>__


In [29]:
def sgd_step(data, x, lam, step):
    (a, i, r) = data
    (u, b_u, v, b_v) = x
    (lam_u, lam_v) = lam
    print('lam_u', lam_u, 'lam_v', lam_v)
    # Your code here
    u_a = u[a]
    b_u_a = b_u[a]
    lam_u_a = lam_u[a]
    v_i = v[i]
    b_v_i = b_v[i]
    lam_v_i = lam_v[i]
    pred = np.dot(u_a.T, v_i) + b_u_a + b_v_i
    d_u_a = (pred - r) * v_i + lam_u_a * u_a
    d_b_u_a = pred - r
    d_v_i = (pred - r) * u_a + lam_v_i * v_i
    d_b_v_i = pred - r
    x[0][a] = u_a - step*d_u_a
    x[1][a] = b_u_a - step*d_b_u_a
    x[2][i] = v_i - step*d_v_i
    x[3][i] = b_v_i - step*d_b_v_i
    return x

# def sgd_step(data, x, lam, step):
#     (a, i, r) = data
#     (u, b_u, v, b_v) = x
#     (lam_u, lam_v) = lam
#     print('lam_u', lam_u, 'lam_v', lam_v)
#     print('lam_u.shape', lam_u.shape, 'lam_v.shape', lam_v.shape)
#     # Your code here
#     u_a = u[a]
#     b_u_a = b_u[[a]].reshape((1, 1))
#     lam_u_a = lam_u[[a]].reshape((1, 1))
#     v_i = v[i]
#     b_v_i = b_v[[i]].reshape((1, 1))
#     lam_v_i = lam_v[[i]].reshape((1, 1))
#     print('u_a', u_a, 'b_u_a', b_u_a, 'lam_u_a', lam_u_a, 'v_i', v_i, 'b_v_i', b_v_i, 'lam_v_i', lam_v_i)
#     print('u_a.shape', u_a.shape, 'b_u_a.shape', b_u_a.shape, 'lam_u_a.shape', lam_u_a.shape, 'v_i.shape', v_i.shape, 'b_v_i.shape', b_v_i.shape, 'lam_v_i.shape', lam_v_i.shape)
#     d_u_a = (np.dot(u_a.T, v_i) + b_u_a + b_v_i - r) * v_i + lam_u_a * u_a
#     d_b_u_a = np.dot(u_a.T, v_i) + b_u_a + b_v_i - r
#     print('d_u_a', d_u_a, 'd_b_u_a', d_b_u_a)
#     print('d_u_a.shape', d_u_a.shape, 'd_b_u_a.shape', d_b_u_a.shape)
#     d_v_i = (np.dot(u_a.T, v_i) + b_u_a + b_v_i - r) * u_a + lam_v_i * v_i
#     d_b_v_i = np.dot(u_a.T, v_i) + b_u_a + b_v_i - r
#     print('d_v_i', d_v_i, 'd_b_v_i', d_b_v_i)
#     print('d_v_i.shape', d_v_i.shape, 'd_b_v_i.shape', d_b_v_i.shape)
#     x[0][a] = u_a - step*d_u_a
#     x[1][a] = b_u_a - step*d_b_u_a
#     x[2][i] = v_i - step*d_v_i
#     x[3][i] = b_v_i - step*d_b_v_i
#     return x

Here is a function to help test your code. It uses `ratings_small`, defined above. 

In [30]:
def sgd_step_test():
    '''
    This is a test function provided to help you debug your implementation
    '''
    step = 0.025
    lam =(np.array([ 0.00333333,  0.005,  0.00333333,  0.005,  0.00333333]), np.array([ 0.0025,  0.00333333,  0.01,  0.002]))
    np.random.seed(0)
    
    first = []
    for i in range(5):
        first.append(np.random.rand(2, 1))
    second = np.zeros((5,))
    third = []
    for i in range(5):
        third.append(np.random.rand(2, 1))
    fourth = np.zeros((5,))
    x0 = (first, second, third, fourth)
    
    x_result = sgd_step(ratings_small[3], x0, lam, step)
    
    assert np.all(np.isclose(x_result[0], np.array([[[0.5488135039273248], [0.7151893663724195]],
                                                [[0.6667107015911342], [0.5875840438721468]],
                                                [[0.4236547993389047], [0.6458941130666561]],
                                                [[0.4375872112626925], [0.8917730007820798]],
                                                [[0.9636627605010293], [0.3834415188257777]]])))
    assert np.all(np.isclose(x_result[1].reshape(-1,), np.array([[0.0], [0.08086477989447478], [0.0], [0.0], [0.0]]).reshape(-1,)))
    assert np.all(np.isclose(x_result[2], np.array([[[0.8404178830022684], [0.5729237224816648]],
                                                [[0.5680445610939323], [0.925596638292661]],
                                                [[0.07103605819788694], [0.08712929970154071]],
                                                [[0.02021839744032572], [0.832619845547938]],
                                                [[0.7781567509498505], [0.8700121482468192]]])))
    assert np.all(np.isclose(x_result[3].reshape(-1,), np.array([[0.08086477989447478], [0.0], [0.0], [0.0], [0.0]]).reshape(-1,)))
    print("Test passed!")

sgd_step_test()

Test passed!


# 4) MovieLens

Now we're going to start working with some real-world data. There's a
commonly used <a
href="https://grouplens.org/datasets/movielens/">dataset</a> of
movie ratings, called the MovieLens Dataset.

Below, we have included the following utility functions:

* `load_ratings_data_small(path_data='ratings.csv')` returns
  a list of ratings triples (a, i, r) for a subset of the data, to be
  used in parameter tuning.

* `load_ratings_data(path_data='ratings.csv')` returns the full list
  of ratings triples.

* `load_movies(path_movies='movies.csv')` returns two dictionaries.
  The first maps movie indices to title strings and the second maps
  movie indices to a list of genre strings.

In [31]:
def load_ratings_data_small(path_data='hw12_code_and_data/ratings.csv'):
    """
    Returns two lists of triples (a, i, r) (training, validate)
    """
    # we want to "randomly" sample but make it deterministic
    def user_hash(uid):
        return 71 * uid % 401
    def user_movie_hash(uid, iid):
        return (17 * uid + 43 * iid) % 61
    data_train = []
    data_validate = []
    with open(path_data) as f_data:
        for line in f_data:
            (uid, iid, rating, timestamp) = line.strip().split(",")
            h1 = user_hash(int(uid))
            if h1 <= 40:
                h2 = user_movie_hash(int(uid), int(iid))
                if h2 <= 12:
                    data_validate.append([int(uid), int(iid), float(rating)])
                else:
                    data_train.append([int(uid), int(iid), float(rating)])
    print('Loading from', path_data,
          'users_train', len(set(x[0] for x in data_train)),
          'items_train', len(set(x[1] for x in data_train)),
          'users_validate', len(set(x[0] for x in data_validate)),
          'items_validate', len(set(x[1] for x in data_validate)))
    return data_train, data_validate

def load_ratings_data(path_data='hw12_code_and_data/ratings.csv'):
    """
    Returns a list of triples (a, i, r)
    """
    data = []
    with open(path_data) as f_data:
        for line in f_data:
            (uid, iid, rating, timestamp) = line.strip().split(",")
            data.append([int(uid), int(iid), float(rating)])

    print('Loading from', path_data,
          'users', len(set(x[0] for x in data)),
          'items', len(set(x[1] for x in data)))
    return data

def load_movies(path_movies='hw12_code_and_data/movies.csv'):
    """
    Returns a dictionary mapping item_id to item_name and another dictionary
    mapping item_id to a list of genres
    """
    data = {}
    genreMap = {}
    with open(path_movies, encoding = "utf8") as f_data:
        for line in f_data:
            parts = line.strip().split(",")
            item_id = int(parts[0])
            item_name = ",".join(parts[1:-1]) # file is poorly formatted
            item_genres = parts[-1].split("|")
            data[item_id] = item_name
            genreMap[item_id] = item_genres
    return data, genreMap

You will need to use the above functions to answer the questions below.

**NOTE: Our checkers will succeed on 95% of submissions stemming from
correct implementations.  However, it is possible (but unlikely) for
your implementation to be correct but your answer to be rejected due
to getting an unlucky initialization.  If you are fairly sure that
your code is correct, you should try re-training a new model and
submitting answers from the new model.**

If you haven't already, complete the implementations of `update_U`,
`update_V`, and `sgd_step` (if you implemented it) above based on your solutions
to the previous questions; the definition of `update_V` can be found via View Answer in
the `update_U` problem.  We will limit ourselves to running ALS in
this problem since it requires less parameter tuning than SGD does.

## 4.1) Recommendations

In the following, we will want to be able to save models locally so as to avoid running als (which takes some time) too many times. In order to do this, we include the following helper functions:


In [32]:
# After retrieving the output x from mf_als, you can use this function to save the output so
# you don't have to re-train your model
def save_model(x):
    pickle.dump(x, open("ALSmodel", "wb"))

# After training and saving your model once, you can use this function to retrieve the previous model
def load_model():
    x = pickle.load(open("ALSmodel", "rb"))
    return x

Now, compute a model with ALS and save it to disk, by executing the following code:

<b> Note: If you are running into errors when running `mf_als`, make sure that your code for `update_U` did not update a user's entries if that user had no ratings. </b>

In [33]:
data = load_ratings_data()
movies_dict, genres_dict = load_movies()
model = mf_als(data, None, k=10, lam=1, max_iter=20)
save_model(model)

Loading from hw12_code_and_data/ratings.csv users 13366 items 2000
Iteration 1 finished. Total Elapsed Time: 3.10
Iteration 2 finished. Total Elapsed Time: 6.13
Iteration 3 finished. Total Elapsed Time: 9.24
Iteration 4 finished. Total Elapsed Time: 12.30
Iteration 5 finished. Total Elapsed Time: 15.35
Iteration 6 finished. Total Elapsed Time: 18.35
Iteration 7 finished. Total Elapsed Time: 21.38
Iteration 8 finished. Total Elapsed Time: 24.39
Iteration 9 finished. Total Elapsed Time: 27.42
Iteration 10 finished. Total Elapsed Time: 30.45
Iteration 11 finished. Total Elapsed Time: 33.49
Iteration 12 finished. Total Elapsed Time: 36.48
Iteration 13 finished. Total Elapsed Time: 39.54
Iteration 14 finished. Total Elapsed Time: 42.55
Iteration 15 finished. Total Elapsed Time: 45.59
Iteration 16 finished. Total Elapsed Time: 48.59
Iteration 17 finished. Total Elapsed Time: 51.61
Iteration 18 finished. Total Elapsed Time: 54.62
Iteration 19 finished. Total Elapsed Time: 57.68
Iteration 20 f

This takes two minutes or so on a reasonably fast laptop. As alluded to before, <b>you only have to run `mf_als` once</b> -- henceforth, you can load the saved model via:

In [34]:
model = load_model()

Note that `load_model()` returns a tuple $(u, b_u, v, b_v)$ corresponding to the matrix factorization learned using the alternating least squares algorithm from above.

We will assume this model for the rest of the questions below.

We have introduced a "synthetic" user into the ratings file with id = 270894. Remember that we can access user data in `data`, which is a list of $(a, i, r)$ tuples


### 4.1A)
Write a piece of code to get relevant movie ratings in `data`, then look for the movies in `genres_dict`. Based on the movies that they've rated 5.0, what is this user's favorite genre? A list of all possible genres is given by the `genres` list below:

In [36]:
genres = ['Western', 'Comedy', 'Children', 'Crime', 'Musical',
          'Adventure', 'Drama', 'Horror', 'War', 'Documentary',
          'Romance', 'Animation', 'Film-Noir', 'Sci-Fi', 'Mystery',
          'Fantasy', 'IMAX', 'Action', 'Thriller']

# Your code to find the user's favorite genre here:
user_id = 270894
users_genre_preference = {}
for a, i, r in data:
    if a != user_id or r != 5.0:
        continue
    movie_genres = genres_dict[i]
    for genre in movie_genres:
        users_genre_preference[genre] = users_genre_preference.get(genre, 0) + 1
print(max(users_genre_preference.items(), key=lambda x: x[1]))

('Animation', 74)


### 4.1B)
Write a piece of code to find the top 50 movies in `movies_dict` that are predicted for this user.  How many movies in this list match their favorite genre? Remember to remove movies the user has already seen.

In [70]:
#Your code to find how many movies match their favorite genre here
def pred(a, i, model):
    (u, b_u, v, b_v) = model
    return np.dot(u[a].T, v[i]) + b_u[a] + b_v[i]

user_id = 270894
all_movies = set([i for i in movies_dict])
movies_seen_by_user = set([i for a, i, _ in data if a == user_id])
movies_not_seen_by_user = all_movies.difference(movies_seen_by_user)
predictions = [(user_id, i, pred(user_id, i, model)[0, 0]) for i in movies_not_seen_by_user]
predictions.sort(key=lambda x: x[2], reverse=True)
animation_predicted_top_50 = list(filter(lambda x: 'Animation' in genres_dict[x[1]], predictions[:50]))
print(animation_predicted_top_50)
print(len(animation_predicted_top_50))

[(270894, 78499, 5.569226831622767), (270894, 2761, 5.35335081092243), (270894, 588, 5.339685230759691), (270894, 6377, 5.332275791924108), (270894, 4886, 5.296043424778638), (270894, 152081, 5.274982107114708), (270894, 720, 5.271315885886009), (270894, 68954, 5.262040271288446), (270894, 745, 5.200077200356188), (270894, 5444, 5.182299512683893), (270894, 1223, 5.18002614608879), (270894, 551, 5.126914094909444), (270894, 5618, 5.046585355720123), (270894, 1033, 4.959269004715457), (270894, 2085, 4.956644834568388), (270894, 616, 4.953235599395153), (270894, 2394, 4.949630999424179), (270894, 106696, 4.921827375048215), (270894, 115617, 4.876607191857276), (270894, 783, 4.868274036695403), (270894, 1023, 4.845080741061674), (270894, 79091, 4.838588311642232), (270894, 103141, 4.782489337676313), (270894, 2089, 4.7712221505534425), (270894, 1566, 4.758214283714145), (270894, 1688, 4.7324014912214345)]
26


## 4.2) Similarity

Typically, movie similarities are estimated based on similarity of
genre or cast, or by human-generated recommendations.  It turns out that
training a recommender system gives us, as a by-product, a new way of
measuring similarity between movies.

First, let's think about what movie similarity might mean.  One
intuition is that two movies A and B are similar if lots of people
like both of them.  However, this criterion can be misleading.  Movies
with high average ratings will look similar since lots of people will tend to
like them.  So we will refine our criterion to be: "If users tend to
like movies A and B more than what is average for the movie and for
the user, then A and B are similar".

### 4.2A)
How can we tell if user $a$ liked movie $i$ more than average (for the movie and the user)? Recall that the predicted rating will be $ u^{(a)}\cdot v^{(i)} + b_u^{(a)} + b_v^{(i)}$. Under what conditions will this predicted rating be higher than average for the movie and the user? Hint: Are the biases $b_u^{(a)}$ and/or $b_v^{(i)}$ relevant? Are the magnitudes of $u^{(a)}$ and/or $v^{(i)}$ relevant?

We can generalize the above criterion to be "If users tend to rate movies A and B similarly relative to the average, then A and B are similar". Our previous criterion
only discussed the case where ratings are high, but this new criterion should hold for low 
ratings as well, for example. So in general, if movies A and B are similar, then for
every user $a$ we should expect the dot product/angle (based on the answer to the previous question)
between $u^{(a)}$ and $v^{(A)}, v^{(B)}$ to be very close. This in turn implies that the two vectors
$v^{(A)}, v^{(B)}$ are either very close together in space or very close in angle, again
depending on the answer to the previous question. This motivates us to use <a
href="https://en.wikipedia.org/wiki/Cosine_similarity">cosine
similarity</a> to compute similarities between movies, being the standard way
to measure the type of proximity we are looking for. For the rest of
this homework the similarity between movies $A$ and $B$ will be
computed as $\frac{v^{(A)} \cdot v^{(B)}}{\|v^{(A)}\| \|v^{(B)}\|}$.
**Note that the similarity can be positive or negative.**

Write a piece of code which identifies the movies in the dataset with
the highest similarity to a given movie, using the formula for cosine
similarity defined above.

### 4.2B)
Find the 10 movies (in `movies_dict`) most similar to "Star Wars: Episode IV - A New Hope (1977)" (id 260).

### 4.2C)
Find the 10 movies (in `movies_dict`) most similar to "Star Wars: Episode I - The Phantom Menace (1999)" (id 2628).

In [109]:
#Code to identify movies with highest similarity to a given movie
def similarity(i, j, model):
    u, b_u, v, b_v = model
    return np.dot(v[i].T, v[j]) / (np.linalg.norm(v[i]) * np.linalg.norm(v[j]))

def compute_similarity_scores(i, movies_dict):
    similarity_scores = []
    for j in movies_dict:
        if i == j:
            continue
        similarity_score = similarity(i, j, model)
        similarity_scores += [(j, similarity_score)]
    return similarity_scores
        
movie_id = 260
movie_similarity_scores = compute_similarity_scores(movie_id, movies_dict)
movie_similarity_scores.sort(key=lambda x: x[1], reverse=True)
top_10_similar_movies = list(map(lambda x: x[0], movie_similarity_scores[:10]))
print(top_10_similar_movies)
print(list(map(lambda x: movies_dict[x], top_10_similar_movies)))

movie_id = 2628
movie_similarity_scores = compute_similarity_scores(movie_id, movies_dict)
movie_similarity_scores.sort(key=lambda x: x[1], reverse=True)
top_10_similar_movies = list(map(lambda x: x[0], movie_similarity_scores[:10]))
print(top_10_similar_movies)
print(list(map(lambda x: movies_dict[x], top_10_similar_movies)))

[1196, 1210, 1198, 1291, 2947, 2993, 592, 122886, 33493, 6104]
['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Indiana Jones and the Last Crusade (1989)', 'Goldfinger (1964)', 'Thunderball (1965)', 'Batman (1989)', 'Star Wars: Episode VII - The Force Awakens (2015)', 'Star Wars: Episode III - Revenge of the Sith (2005)', 'Monty Python Live at the Hollywood Bowl (1982)']
[5378, 33493, 6934, 6365, 59615, 7318, 52722, 153, 1210, 59501]
['Star Wars: Episode II - Attack of the Clones (2002)', 'Star Wars: Episode III - Revenge of the Sith (2005)', '"Matrix Revolutions, The (2003)"', '"Matrix Reloaded, The (2003)"', 'Indiana Jones and the Kingdom of the Crystal Skull (2008)', '"Passion of the Christ, The (2004)"', 'Spider-Man 3 (2007)', 'Batman Forever (1995)', 'Star Wars: Episode VI - Return of the Jedi (1983)', '"Chronicles of Narnia: Prince Caspi

Make sure that you also print the movie names (using `movies_dict`) so you can see that they
make sense!  These results are fairly interesting; they would be quite
clear to one who has seen the movies as the Star Wars prequels (Episodes
I-III) and the originals (Episodes IV-VI) are very different movies!
But it's interesting that our model can learn this difference.

Now, we look at how similar movies within the same genre are. You can
use `genres_dict` to help with this.

### 4.2D)
For calibration, compute the average similarity between all pairs
of movies in `movies_dict`. **Remember not to compare a movie to itself.**

In [110]:
#Code to compute average similarity between all pairs of movies
def compute_average_similarity_of_all_pairs(movies_dict):
    count = 0
    similarity_score_sum = 0
    all_movies = [i for i in movies_dict]
    for i in range(len(all_movies)):
        for j in range(i+1, len(all_movies)):
            similarity_score_sum += similarity(all_movies[i], all_movies[j], model)
            count += 1
    return similarity_score_sum / count

avg_similarity = compute_average_similarity_of_all_pairs(movies_dict)

Now compute the average similarities across all pairs of movies within
each genre. Remember that a list of all possible genres is given by
the `genres` list defined at the top of your code file.

### 4.2E)
Find the genre whose movies have the highest average similarity and the value of that average similarity.

In [111]:
#Code to find the genre with the highest average similarity and its value
def compute_genre_average_similarity_of_all_pairs(genre_to_movies):
    genre_to_similarity_score = {}
    for genre in genre_to_movies:
        count = 0
        similarity_score_sum = 0
        movies_in_genre = genre_to_movies[genre]
        for i in range(len(movies_in_genre)):
            for j in range(i+1, len(movies_in_genre)):
                similarity_score_sum += similarity(movies_in_genre[i], movies_in_genre[j], model)
                count += 1
        genre_to_similarity_score[genre] = similarity_score_sum / count
    return genre_to_similarity_score
    

genre_to_movies = {genre : [] for genre in genres}
for i in movies_dict:
    movie_genres = genres_dict[i]
    for genre in movie_genres:
        genre_to_movies[genre] += [i]
        
genre_to_similarity_score = compute_genre_average_similarity_of_all_pairs(genre_to_movies)
print(max(genre_to_similarity_score.items(), key=lambda x: x[1]))

('Film-Noir', array([[0.43176651]]))


### 4.2F)
Find the genre whose movies have the lowest average similarity and the value of that average similarity.

In [105]:
#Code to find the genre with the lowest average similarity and its value
print(min(genre_to_similarity_score.items(), key=lambda x: x[1]))

('Crime', array([[0.05645273]]))


These results tell us that there is enormous variation in how similar
the movies under most genres are.  But, note that the similarities are
generally *positive* for movies in a genre.

Next we look at the similarities across genres. More specifically, as we compute the similarity of different genres to the Comedy genre, we find the average similarity of pairs of movies, where one movie belongs to the Comedy genre and the other belongs to the genre being compared to. Note that movies can belong to multiple genres, so it's possible that both movies technically belong to the Comedy genre.

### 4.2G)
Which genre is most similar to Comedy?

In [116]:
#Code to find the genre most similar to Comedy
def compute_genre_to_genre_average_similarity(target_genre, genre_to_movies):
    genre_to_similarity_score = {}
    movies_in_target_genre = genre_to_movies[target_genre]
    for genre in genre_to_movies:
        if genre == target_genre:
            continue
        count = 0
        similarity_score_sum = 0
        movies_in_genre = genre_to_movies[genre]
        for i in range(len(movies_in_target_genre)):
            for j in range(len(movies_in_genre)):
                similarity_score_sum += similarity(movies_in_target_genre[i], movies_in_genre[j], model)
                count += 1
        genre_to_similarity_score[genre] = similarity_score_sum / count
    return genre_to_similarity_score

genre_to_movies = {genre : [] for genre in genres}
for i in movies_dict:
    movie_genres = genres_dict[i]
    for genre in movie_genres:
        genre_to_movies[genre] += [i]

target_genre = 'Comedy'
target_genre_to_similarity_score = compute_genre_to_genre_average_similarity(target_genre, genre_to_movies)
print(max(target_genre_to_similarity_score.items(), key=lambda x: x[1]))

('Children', array([[0.04526065]]))


### 4.2H)
Which genre is least similar to Comedy?

In [117]:
#Code to find genre least similar to Comedy
print(min(target_genre_to_similarity_score.items(), key=lambda x: x[1]))

('War', array([[-0.06082649]]))


Observe that this last similarity value is negative.

# 5) Collaborative filtering and the SVD (OPTIONAL)
As an exercise to explore the use of SVD for collaborative filtering, go back to the dataset with
$$
Y =
\begin{bmatrix}
    4 & 3 & 1 \\
    1 & 3 & -2 \\
    5 & 2 & 3 \\
\end{bmatrix}
$$
and write some python code to compute $U$ and $V^T$ using the SVD.

In [122]:
Y = np.array([[4, 3, 1], [1, 3, -2], [5, 2, 3]])

# Compute the truncated SVD of Y
U, s, Vt = np.linalg.svd(Y, full_matrices=False)
k = 2
U = U[:, :k]
s = s[:k]
Vt = Vt[:k, :]

# Construct the diagonal matrix Sigma
Sigma = np.diag(s)

# Verify that Y = U @ Sigma @ Vt
Y_reconstructed = U @ Sigma @ Vt
print("Y = \n", Y)
print("U @ Sigma @ Vt = \n", Y_reconstructed)
print(U, Vt)

Y = 
 [[ 4  3  1]
 [ 1  3 -2]
 [ 5  2  3]]
U @ Sigma @ Vt = 
 [[ 4.  3.  1.]
 [ 1.  3. -2.]
 [ 5.  2.  3.]]
[[-0.6311939  -0.19072721]
 [-0.21351391 -0.88911878]
 [-0.74565815  0.41604197]] [[-0.80844891 -0.50325864 -0.30519027]
 [ 0.11435482 -0.64295988  0.75731471]]
